In [30]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas

In [31]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [32]:
url = "https://www.amazon.eg/s?k=%D9%82%D8%B7%D8%B7&crid=1W9DGJBWD56SO&sprefix=%D9%82%D8%B7%2Caps%2C169&ref=nb_sb_noss_2"
response = requests.get(url, headers={
    'User-Agent': 'Mozilla 5 (Educational)'
})
print('contains listitem is ', response.text.count('listitem'))


contains listitem is  72


In [33]:
soup = BeautifulSoup(response.text, 'html.parser')
listitems = soup.find_all('div', attrs={
    'role':'listitem'
})
print(len(listitems))
print(listitems[0])

72
<div class="sg-col-4-of-4 sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-8 sg-col-4-of-20" data-asin="B0CTSR7CM2" data-component-type="s-search-result" data-index="2" data-uuid="dc598214-f603-4bd8-bf20-0730daaae5da" id="dc598214-f603-4bd8-bf20-0730daaae5da" role="listitem"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-2" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-csa-c-item-id="amzn1.asin.1.B0CTSR7CM2" data-csa-c-pos="1" data-csa-c-type="item" data-csa-op-log-render="">
<div class="rush-component s-expand-height" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-eu.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTracking.prod?qualifier=1769615642&amp;id=7704541110584768&amp;widgetName=sp_atf&amp;a

In [34]:
def clean_text(s):
    return re.sub(r'[\u200e\u200f\xa0]', ' ', s).strip()

In [40]:
items = []
for listitem in listitems:
    item = {}
    
    # Title
    title = listitem.find('div', attrs={
        'data-cy':'title-recipe'
    })
    if title:
        item['title'] = title.text
        # remove ADs
        if item['title'].find('إعلان') != -1:
            print('-----SKIPPED-----', title.text)
            continue

    # Price
    price = listitem.find('div', attrs={
        'data-cy':'price-recipe'
    })
    if(price):
        symbol = clean_text(price.select_one('span.a-price-symbol').text)
        whole = clean_text(price.select_one('span.a-price-whole').text)
        fraction = clean_text(price.select_one('span.a-price-fraction').text)
        # print('whole', whole)
        # print('fraction', fraction)
        priceText = f'{whole}{fraction}'
        priceText = priceText.replace(' ', '')
        priceText = priceText.replace(',', '')
        item['price'] = float(priceText)
        item['currency'] = symbol
    
    # Badge (ex: Best seller)
    badge = listitem.select_one('div#amazonBadges-capsule-view div div div')
    if badge:
        # inside inside the badge (ex: id="BEST_SELLER" inside it) 
        # .. assuming one badge for each .. if more 1 is taken
        badgeParts = list(badge.children)
        badgeTitle = badgeParts[0].text
        badgeAmong = badgeParts[1].text
        item['badge'] = badgeTitle
        item['badge among'] = badgeAmong

    # Image
    img = listitem.find('div', attrs={
        'data-cy': 'image-container'
    })
    if(img):
        img = img.select_one('img')
    if(img):
        item['image'] = img.attrs['src']
    else:
        item['image'] = None
    # print(item.get('image'))

    # remove Empty
    if not title or not price:
        continue

    items.append(item)
    print(item)  

-----SKIPPED----- إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  باب أمان رفيع منزلق مقاوم للطقس للقطط والحيوانات الأليفة وللنوافذ والأبواب مع قفل رباعي الاتجاهات من تراسل، مقاس كبير 
-----SKIPPED----- إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  SARAR دراي فود للقطط المعقمة يوريناري كير بنكهة السلمون والتوت البري | 30% بروتين و12% دهون | دعم الكلى والجهاز البولي | 2 كجم 
-----SKIPPED----- إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  طعام جاف فاخر للقطط الكبيرة من ايكو فارم - 10 كجم 
-----SKIPPED----- إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  طعام جاف للقطط البالغة S/O للعناية بالجهاز البولي للقطط البالغة من كيه 9، 2 كجم 
{'title': 'فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، فرشاة بخار تنظيف ذاتي للتدليك، تزيل الشعر الطائر المتناثر، فرشاة إزالة ال

In [41]:
df = pd.DataFrame(items)

In [42]:
print(df)

                                                title   price currency  \
0   فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، ف...  106.00     جنيه   
1   لوح خدش للقطط والحيوانات الأليفة من الورق المق...  149.00     جنيه   
2                            كرة النعناع البري للقطط    55.00     جنيه   
3   طعام رطب للقطط فريسكيز بقطع اللحم البقري في ال...   39.60     جنيه   
4   رمل فضلات القطط سريع التكتل ومضاد للبكتيريا وا...   85.00     جنيه   
5   طعام مجفف من رويال كات بطعم الدجاج الحقيقي للق...  225.00     جنيه   
6   طعام رطب للقطط بنكهة الدجاج من باوير، 400 جرام...   50.00     جنيه   
7   رمل متكتل لفضلات القطط من دودزي، 5 لتر (بنفسجي...  139.00     جنيه   
8   قصافة أظافر الحيوانات الأليفة الصغيرة - مقص اح...   79.00     جنيه   
9   سجادة خدش للقطط - لوح خدش للقطط - وسادة مريحة ...  169.00     جنيه   
10  وعاء مزدوج للقطط مزود بشرب أوتوماتيكي، وعاء تغ...  230.00     جنيه   
11  SKEIDO سرير كلب للقطط دافئ دافئ بيت الكلب النا...  199.00     جنيه   
12  اوعية طعام وشراب مزدوجة للقطط والك

In [43]:
df.describe(include='all')

,title,price,currency,badge,badge among,image
count,48,48.00000,48,10,10,48
unique,48,NaN,1,1,9,48
top,فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، ف...,NaN,جنيه,الأكثر مبيعاً,في Pet Supplies,https://m.media-amazon.com/images/I/61dopS6y4B...
freq,1,NaN,48,10,2,1
mean,NaN,160.61000,NaN,NaN,NaN,NaN
std,NaN,121.69278,NaN,NaN,NaN,NaN
min,NaN,39.60000,NaN,NaN,NaN,NaN
25%,NaN,81.25000,NaN,NaN,NaN,NaN
50%,NaN,122.00000,NaN,NaN,NaN,NaN
75%,NaN,225.00000,NaN,NaN,NaN,NaN


In [45]:
df.to_csv('amazon.csv')